<a href="https://colab.research.google.com/github/suomimasuda/solaredgemonitor/blob/main/SolarEdgeMonitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# 現在のディレクトリのファイルを表示
print(os.listdir())

['.config', 'i-s3pv-3e280c26d353.json', 'sample_data']


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 認証情報のセットアップ
def authorize_google_sheets():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/Colab Notebooks/i-s3pv-3e280c26d353.json", scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets に接続
client = authorize_google_sheets()
sheet = client.open("SolarEdge_Data").worksheet("SolarData")

# 確認のために A1 の値を取得
print(sheet.acell('A1').value)


None


In [2]:
import urllib.parse
import urllib.request
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime  # datetimeモジュールを追加

# Google Sheets API の認証設定
def authorize_google_sheets():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("/content/i-s3pv-3e280c26d353.json", scope)
    client = gspread.authorize(creds)
    return client

# 除外するサイトIDを取得
def get_excluded_sites(sheet):
    try:
        worksheet = sheet.worksheet("Ex")  # Exシートを取得
        excluded_sites = worksheet.col_values(1)[1:]  # A列のデータを取得（1行目はヘッダー）
        return set(excluded_sites)
    except gspread.exceptions.WorksheetNotFound:
        return set()

# SolarEdge API から発電所リストを取得
def get_site_list(api_key):
    url = 'https://monitoringapi.solaredge.com/sites/list?'
    params = {
        'size': '100',
        'sortProperty': 'name',
        'sortOrder': 'ASC',
        'api_key': api_key
    }
    param_str = urllib.parse.urlencode(params)
    response = urllib.request.urlopen(url + param_str).read()
    return json.loads(response)

# 取得したデータをGoogle Sheetsに保存
def save_sites_to_google_sheets(sheet_name, data):
    client = authorize_google_sheets()
    sheet = client.open("SolarEdge_Data")
    excluded_sites = get_excluded_sites(sheet)

    try:
        worksheet = sheet.worksheet(sheet_name)  # 既存のシートを取得
        worksheet.clear()  # 初期化
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sheet.add_worksheet(title=sheet_name, rows="100", cols="4")  # シートがなければ作成, コラム数を4へ変更

    # ヘッダーを追加
    worksheet.update("A1", [["Site Name", "Site ID", "Alert Quantity", "Execution Time"]]) #ヘッダーを追加

    # データを整形して、除外リストにないもののみ追加
    values = [[site['name'], site['id'], site.get('alertQuantity', 0), datetime.now().strftime('%Y-%m-%d %H:%M:%S')] #実行時間を追加
              for site in data['sites']['site'] if str(site['id']) not in excluded_sites]
    worksheet.update("A2", values)

# 実行部分
if __name__ == "__main__":
    API_KEY = "6PZMKL7XLLR40Q10JKHG8NA7H7MOUNLW"
    site_list_data = get_site_list(API_KEY)
    save_sites_to_google_sheets("SiteList", site_list_data)
    #print("発電所リストとアラート情報をGoogle Sheetsに保存しました。（除外リスト適用済み）")

<ipython-input-2-a777dfebfba3>:50: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("A1", [["Site Name", "Site ID", "Alert Quantity", "Execution Time"]]) #ヘッダーを追加
<ipython-input-2-a777dfebfba3>:55: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("A2", values)


In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
import time

def run_colab_notebook():
    # Rerun this cell to authorize access to your Google Drive account
    # This is usually necessary when you restart the runtime or after a long period of inactivity
    auth.authenticate_user() # Re-authenticate to ensure correct credentials are used
    #print("Authentication complete. You may continue now.")
    # Rest of your code
    service = build('drive', 'v3')
    notebook_id = "1jArZ0tff75qtigWAhZfw6PCKT67-4JZq#scrollTo=qe48u_P75fDv"  # あなたのColabノートブックのIDに変更

    # 実行リクエスト
    execute_url = f"https://colab.research.google.com/drive/{notebook_id}"
    #print(f"Colabノートブックを実行中: {execute_url}")

run_colab_notebook()